### Objectives

First, create a graph representation of the information we have. 
This would be information about the elements, and the nodes. 
Using this, we will feed it into our SVM, or a Random Forest, and see from there onwards

In [177]:
%load_ext autoreload 
%autoreload 2

In [119]:
from Generate_Excel import *
from MitoCode_Functions import open_file_dialog

In [178]:
import pandas as pd
import igraph as ig

def Mitograph_Graph(FileName):
    # Read information from the gnet file, and create a graph object
    # generate a function that takes in the filename and gives out the graph
    G = pd.read_table(FileName + '.gnet', skiprows=1, sep='\t', names=['Source', 'Target', 'Length'])
    graph = ig.Graph.TupleList(G.itertuples(index=False), directed=False, edge_attrs=['Length'])
    layout = graph.layout_auto()  # Automatic layout calculation
    graph.vs['degree'] = graph.degree()
    # ig.plot(graph, layout=layout)
    Vol = pd.read_table(FileName + '.cc', skiprows=0, sep='\t')
    Vol.columns = ['node', 'cc', 'vol_cc']

    ids = [int(node['name']) for node in graph.vs]
    graph.vs['cc_vol'] = [Vol.loc[Vol['node'] == node, 'vol_cc'].values[0] for node in ids]
    graph.vs['cc'] = [Vol.loc[Vol['node'] == node, 'cc'].values[0] for node in ids]
    return graph, Vol

In [179]:


# df_dir_2 = open_file_dialog()[0]
# df = pd.read_csv(df_dir_2)


In [188]:
a, p, c, n, f = Generate_Mito_Tables(open_folder_dialog())

12 folders found. Now running...
csv File generated


In [187]:
f[['folder_name_x']]

,folder_name_x,folder_name_x
0,TCDD1nM1hr5_4_S,TCDD1nM1hr5_4_S
1,TCDD1nM1hr5_4_S,TCDD1nM1hr5_4_S
2,TCDD1nM1hr5_4_S,TCDD1nM1hr5_4_S
3,TCDD1nM1hr5_4_S,TCDD1nM1hr5_4_S
4,TCDD1nM1hr5_4_S,TCDD1nM1hr5_4_S
...,...,...
2103,TCDD1nM1hr1_1_S,TCDD1nM1hr1_1_S
2104,TCDD1nM1hr1_1_S,TCDD1nM1hr1_1_S
2105,TCDD1nM1hr1_1_S,TCDD1nM1hr1_1_S
2106,TCDD1nM1hr1_1_S,TCDD1nM1hr1_1_S


In [183]:
df = f[['line_id', 'cc_x', 'folder_name_x', 'element_pixel_intensity_405', 'cc_pixel_intensity_405', 'branches', 'nodes', 'edges', 'cc_max_PK', 'cc_length_(um)']]

In [184]:
df

,line_id,cc_x,folder_name_x,folder_name_x,element_pixel_intensity_405,cc_pixel_intensity_405,branches,nodes,edges,cc_max_PK,cc_length_(um)
0,290,1,TCDD1nM1hr5_4_S,TCDD1nM1hr5_4_S,19.50,19.50,1,2,1,2,0.87492
1,290,1,TCDD1nM1hr5_4_S,TCDD1nM1hr5_4_S,19.50,19.50,1,2,1,2,0.87492
2,290,1,TCDD1nM1hr5_4_S,TCDD1nM1hr5_4_S,19.50,19.50,1,2,1,2,0.87492
3,290,1,TCDD1nM1hr5_4_S,TCDD1nM1hr5_4_S,19.50,19.50,1,2,1,2,0.87492
4,290,1,TCDD1nM1hr5_4_S,TCDD1nM1hr5_4_S,19.50,19.50,1,2,1,2,0.87492
...,...,...,...,...,...,...,...,...,...,...,...
2103,7,125,TCDD1nM1hr1_1_S,TCDD1nM1hr1_1_S,41.50,41.50,1,2,1,2,0.20800
2104,4,126,TCDD1nM1hr1_1_S,TCDD1nM1hr1_1_S,38.25,38.25,1,2,1,2,0.44123
2105,4,126,TCDD1nM1hr1_1_S,TCDD1nM1hr1_1_S,38.25,38.25,1,2,1,2,0.44123
2106,4,126,TCDD1nM1hr1_1_S,TCDD1nM1hr1_1_S,38.25,38.25,1,2,1,2,0.44123


In [153]:
bigN = pd.concat(n)


In [164]:
groupn = bigN.groupby(['cc', 'folder_name'])
dataframelist = [group for _, group in groupn]

In [155]:
for i in dataframelist: 
    i['node_2'] = i['node'].shift(-1)
    i.dropna(subset=['node_2'], inplace=True)
    
bigDf = pd.concat(dataframelist)


In [189]:
df.drop_duplicates(subset=['cc_x', 'folder_name'], inplace=True)

KeyError: Index(['folder_name'], dtype='object')

In [190]:
df.columns

Index(['line_id', 'cc_x', 'folder_name_x', 'folder_name_x',
       'element_pixel_intensity_405', 'cc_pixel_intensity_405', 'branches',
       'nodes', 'edges', 'cc_max_PK', 'cc_length_(um)'],
      dtype='object')

In [191]:
# drop one of the folder_name_x
df.drop(columns=['folder_name_x'], inplace=True)

/tmp/ipykernel_412419/653725729.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['folder_name_x'], inplace=True)


In [192]:
eledf = df[['line_id', 'cc_x', 'element_Volume_Voxel', 'element_pixel_intensity_405', 'element_length_(um)', 'element_pixel_intensity_488', 'element_pixel_intensity_555' 
            ,'element_pixel_intensity_ratio', 'element_average_width', 'folder_name_x']].drop_duplicates(subset=['line_id', 'folder_name_x'])

# rename folder_name_x to folder_name in eledf
# eledf.rename(columns={'folder_name_x': 'folder_name'}, inplace=True)
bigDf = pd.merge(bigDf, eledf, how='left', left_on=['line_id', 'folder_name'], right_on=['line_id', 'folder_name_x'])

KeyError: "['element_Volume_Voxel', 'element_length_(um)', 'element_pixel_intensity_488', 'element_pixel_intensity_555', 'element_pixel_intensity_ratio', 'element_average_width', 'folder_name_x'] not in index"

In [ ]:
bigDf

,x,y,z,node,line_id,point_id,degree,cc,vol_cc,avg_PK_Of_element,element_connectivity,folder_name,node_2
380,4.3680,3.0160,2.5,244,0,4,3,49,1.70352,2.0,0.500000,TCDD1nM1hr1_1_S,399.0
559,11.4400,12.5840,2.5,306,0,3,1,0,0.00000,1.0,1.000000,TCDD1nM1hr1_2_S,373.0
660,20.1413,2.7733,2.0,430,0,4,3,45,4.99699,2.0,0.500000,TCDD1nM1hr1_3_S,487.0
328,7.5920,1.7680,1.5,209,0,3,1,59,0.04326,1.0,1.000000,TCDD1nM1hr2_1_S,210.0
671,11.7520,1.3520,3.0,410,0,3,1,0,0.00000,1.0,1.000000,TCDD1nM1hr2_2_S,700.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,13.5200,13.0000,4.0,177,547,20,1,70,1.00589,1.0,1.000000,TCDD1nM1hr5_1_S,206.0
178,4.2640,17.5760,4.5,152,548,5,1,42,27.72682,2.5,0.400000,TCDD1nM1hr5_1_S,201.0
200,17.8048,8.6528,4.1,165,549,2,4,29,21.85373,3.5,0.285714,TCDD1nM1hr5_1_S,185.0
112,16.4320,18.8240,6.0,95,550,9,1,49,0.42723,1.0,1.000000,TCDD1nM1hr5_1_S,113.0


In [ ]:
bigDf = bigDf[['line_id', 'folder_name_x', 'node', 'node_2', 'element_pixel_intensity_488', 'element_pixel_intensity_405', 'element_pixel_intensity_555'element_length_(um)', 'element_average_width']]

KeyError: "['folder_name_x', 'element_pixel_intensity_488', 'element_pixel_intensity_405', 'element_pixel_intensity_555', 'element_length_(um)', 'element_average_width'] not in index"

In [ ]:
bigDf

,line_id,folder_name_x,folder_name_x,node,node_2,element_pixel_intensity_488,element_length_(um),element_average_width
0,0,TCDD1nM1hr5_4_S,TCDD1nM1hr5_4_S,369.0,399.0,406,3.293642,0.308990
1,0,TCDD1nM1hr5_2_S,TCDD1nM1hr5_2_S,420.0,399.0,586,1.840017,0.249003
2,0,TCDD1nM1hr5_1_S,TCDD1nM1hr5_1_S,667.0,399.0,503,0.338815,0.290197
3,0,TCDD1nM1hr2_2_S,TCDD1nM1hr2_2_S,700.0,399.0,115,1.563550,0.355040
4,0,TCDD1nM1hr2_3_S,TCDD1nM1hr2_3_S,206.0,399.0,551,1.529454,0.256438
...,...,...,...,...,...,...,...,...
4180475,547,TCDD1nM1hr5_1_S,TCDD1nM1hr5_1_S,206.0,206.0,347,2.877844,0.288166
4180476,548,TCDD1nM1hr5_1_S,TCDD1nM1hr5_1_S,201.0,201.0,516,1.443251,0.327078
4180477,549,TCDD1nM1hr5_1_S,TCDD1nM1hr5_1_S,185.0,185.0,400,2.312396,0.306356
4180478,550,TCDD1nM1hr5_1_S,TCDD1nM1hr5_1_S,113.0,113.0,216,1.534589,0.226927


In [ ]:
# make category element_pixelitnensity)_488 into integer
bigDf['element_pixel_intensity_488'] = bigDf['element_pixel_intensity_488'].astype('int64')

/tmp/ipykernel_412419/3401633684.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bigDf['element_pixel_intensity_488'] = bigDf['element_pixel_intensity_488'].astype('int64')


In [ ]:
bigDf = bigDf.drop(['folder_name_x', 'line_id', 'node', 'node_2'], axis=1)

In [ ]:
bigDf

,element_pixel_intensity_488,element_length_(um),element_average_width
0,406,3.293642,0.308990
1,586,1.840017,0.249003
2,503,0.338815,0.290197
3,115,1.563550,0.355040
4,551,1.529454,0.256438
...,...,...,...
4180475,347,2.877844,0.288166
4180476,516,1.443251,0.327078
4180477,400,2.312396,0.306356
4180478,216,1.534589,0.226927


In [ ]:
graph = ig.Graph.TupleList(bigDf.itertuples(index=False), directed=False, edge_attrs=['line_id', 'element_pixel_intensity_ratio', 'element_pixel_intensity_405', 'element_length_(um)', 'element_pixel_intensity_488'])

# ids = [int(node['name']) for node in graph.vs]
# graph.vs['cc_vol'] = [df.loc[df['node'] == node, 'degree'].values[0] for node in ids]
# graph.vs['488'] = [df.loc[df['node'] == node, 'pixel_intensity_488'].values[0] for node in ids]
# graph.vs['405'] = [df.loc[df['node'] == node, 'pixel_intensity_405'].values[0] for node in ids]
# graph.vs['ratio'] = [df.loc[df['node'] == node, 'pixel_intensity_ratio'].values[0] for node in ids]
# graph.vs['structural'] = [df.loc[df['node'] == node, 'pixel_intensity_555'].values[0] for node in ids]

In [ ]:
corr = bigDf.corr()

In [ ]:
corr

,element_pixel_intensity_488,element_length_(um),element_average_width
element_pixel_intensity_488,1.000000,0.111319,0.414200
element_length_(um),0.111319,1.000000,0.276737
element_average_width,0.414200,0.276737,1.000000


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report


In [ ]:
X = bigDf.drop(['element_pixel_intensity_488', 'node', 'node_2', 'line_id', 'element_pixel_intensity_ratio', 'element_pixel_intensity_405'], axis=1)  # Adjust 'target_column' to your actual target column name
y = bigDf['element_pixel_intensity_488']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
svm_model = SVC(kernel='linear')  # You can choose different kernels like 'rbf', 'poly', etc.
svm_model.fit(X_train, y_train)


SVC(kernel='linear')

In [ ]:
y_pred = svm_model.predict(X_test)


In [ ]:
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)



C:\Users\birat\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\birat\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\birat\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1469: Undefi